In [1]:
library(dplyr)
library(ggplot2)
library(stringr)
library(IRanges)
library(igraph)
library(purrr)
library(tidyr)
library(parallel)
library(pbapply)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: BiocGenerics

Loading required package: generics


Attaching package: ‘generics’


The following object is masked from ‘package:dplyr’:

    explain


The following objects are masked from ‘package:base’:

    as.difftime, as.factor, as.ordered, intersect, is.element, setdiff,
    setequal, union



Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:dplyr’:

    combine


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pm

In [2]:
# Functions #

find_motif_cluster_in_sea <- function(sea_res,mc,full.only=F,min_nmotif=3,btwn_mt_max_gap=30,ncores=4) {
  sea_res_sort <- sea_res %>% group_by(seq_name) %>% arrange(start)
  
  meme_size <- sea_res[!duplicated(sea_res$meme_name),'site_width']
  names(meme_size) <- sea_res[!duplicated(sea_res$meme_name),'meme_name']
  sea_spl <- split(sea_res, sea_res$seq_name)

  # Consecutive meme pairs in mc for consecutive gap check
  consec_mp  <- c()
  for (i in 1:(length(mc)-1)) {
    consec_mp <- c(consec_mp, paste0(names(mc)[i],'_',names(mc)[i+1]))
  }

  # Force progress bar to show
  pboptions(type = "txt")
  
  clst_found <- pblapply(sea_spl,function(x) {
    name <- x$seq_name %>% unique
    # message(Sys.time(),' Processing ', name, ' with ', nrow(x), ' motifs')
    gene_strand <- unique(x$strand)
    x_ranges <- IRanges(start = x$start, width = x$site_width, names = row.names(x))
    hits <- findOverlaps(x_ranges, drop.self = TRUE)
    x_nonovl_combs <- list()
    
    if (length(hits)>0) {
      overlapping_vertices <- unique(c(queryHits(hits), subjectHits(hits)))
      overlapping_motif_ids <- row.names(x)[overlapping_vertices]
      
      edge_list <- data.frame(
        from = row.names(x)[queryHits(hits)],
        to = row.names(x)[subjectHits(hits)]
      )
      
      g <- graph_from_data_frame(edge_list, directed = FALSE, vertices = data.frame(name = overlapping_motif_ids))
      components <- components(g)
      overlapping_groups <- split(names(components$membership), components$membership)
      
      non_overlapping_motifs <- x[!row.names(x) %in% V(g)$name, ]
      
      overlapping_ids_list <- map(overlapping_groups, ~ .x)
      combinations <- cross(overlapping_ids_list)
      
      # message(Sys.time(),' Processing combinations')
      for (i in 1:length(combinations)) {
        # message(Sys.time(), ' Processing combination ', i, '/', length(combinations))
        current_ids <- unlist(combinations[[i]])
        current_df <- rbind(
          non_overlapping_motifs,
          x[row.names(x) %in% current_ids, ]
        )
        current_df <- current_df[order(current_df$site_start),]
        comb_id <- paste(current_df[current_ids,'meme_name'],current_ids,sep='_')
        comb_id <- paste(comb_id, collapse = "_")
        x_nonovl_combs[[paste0("Combination_", comb_id)]] <- current_df
      }
    } else {
      x_nonovl_combs[['Orig_data']] <- x[order(x$site_start),]
    }
    
    clst_found <- matrix(nrow = 0,ncol=8) %>% as.data.frame()
    colnames(clst_found) <- c('combination','seq_name','chr','strand',
                              'meme_start_site_start','meme_end_site_start','meme_seq','ref_motif')
    # clst_found <- list()
    # message('Number of nonovl combinations',': ',length(x_nonovl_combs))
    kidx=1
    for (k in names(x_nonovl_combs)) {
      # message('Processing nonovl combination ',kidx,'/',length(x_nonovl_combs))
      kidx = kidx + 1
      nonovl_comb <- x_nonovl_combs[[k]]
      if (nrow(nonovl_comb)>=min_nmotif) {
        widest_ws <- min(length(mc),nrow(nonovl_comb))
        for (window_size in widest_ws:min_nmotif) {
          windows <- list()
          j=1
          for (i in seq(1,nrow(nonovl_comb)-window_size+1)) {
            slide = nonovl_comb[i:(i+window_size-1),]
            no_duplicate = !(duplicated(slide$meme_name) %>% any())
            if (no_duplicate){
              windows[[j]] <- slide
              j = j+1
            }
          }
          if (length(windows) >= 1) {
            for (i in seq(length(windows))){
              slide = windows[[i]]
              if (gene_strand=='+') {
                coord_sorted <- slide[order(slide$start),]
                coord_sorted <- coord_sorted %>% mutate(
                                gap_to_next = ifelse(row_number() < n(), 
                                                      lead(start) - (end), NA),
                                gap_description = ifelse(row_number() < n(),
                                                      paste0(meme_name, "_", lead(meme_name)),NA))
              } else {
                coord_sorted <- slide[order(slide$start,decreasing = T),]
                coord_sorted <- coord_sorted %>% mutate(
                                gap_to_next = ifelse(row_number() < n(), 
                                                      lag(start) - (end), NA),
                                gap_description = ifelse(row_number() < n(),
                                                      paste0(meme_name, "_", lag(meme_name)),NA))
              }
              consec_mp_gaps <- coord_sorted[coord_sorted$gap_description %in% consec_mp,
                                            ]$gap_to_next
              if (any(consec_mp_gaps > btwn_mt_max_gap, na.rm = T)) {next}

              meme_seq <- coord_sorted$meme_name
              # Flip cluster motifs if strands of corresponding motifs not match
              a <- coord_sorted$site_strand[1]
              b <- mc[coord_sorted$meme_name[1]] %>% unname()
              if (a != b) {
                mc <- mc[length(mc):1]
                mc <- ifelse(mc=='+','-','+')
              }
              
              mc_order <- 1:length(mc)
              names(mc_order) <- names(mc)
              
              order <- mc_order[meme_seq] %>% unname()
              
              order_check <- identical(order,sort(order))
              strandness_check <- identical(coord_sorted$site_strand,unname(mc[meme_seq]))
              
              if (order_check & strandness_check) {
                gaps <- data.frame(meme_start = order[which(abs(diff(order))>1)], 
                                   meme_end = order[which(abs(diff(order))>1)+1])
                if (nrow(gaps)!=0) {
                  gaps <- apply(gaps,1,function(x){
                    mm <- mc_order[(x[1]+1):(x[2]-1)] %>% names()
                    mm_size <- sum(meme_size[mm])
                    meme_start <- mc_order[mc_order %in% x[1]] %>% names()
                    meme_end <- mc_order[mc_order %in% x[2]] %>% names()
                    gap_size <- coord_sorted$site_start[which(coord_sorted$meme_name == meme_end)] - 
                      (coord_sorted$site_start[which(coord_sorted$meme_name == meme_start)] + 
                         coord_sorted$site_width[which(coord_sorted$meme_name == meme_start)])
                    mm <- paste0(mm,collapse = ',')
                    df <- data.frame(gap_start = meme_start, gap_end = meme_end, missing_motifs = mm,
                                     missing_motifs_size = mm_size, gap_size = gap_size)
                    return(df)
                  })
                  gaps <- do.call('rbind',gaps)
                } else {
                  gaps <- matrix(ncol = 5, nrow = 0) %>% as.data.frame()
                  colnames(gaps) <- c('gap_start','gap_end','missing_motifs','missing_motifs_size','gap_size')
                }
                
                res <- data.frame(combination = k,
                                  seq_name = name,
                                  chr = unique(coord_sorted$chr),
                                  strand = gene_strand,
                                  meme_start_site_start = coord_sorted$site_start[1],
                                  meme_end_site_start = coord_sorted$site_start[length(coord_sorted$start)],
                                  meme_seq = paste0(meme_seq,collapse = ','),
                                  ref_motif = paste0(names(mc),collapse = ','),
                                  min_consec_gap = coord_sorted$gap_to_next %>% min(na.rm = T))
                # if (name %in% seq_uniq_in_meme) {print(res)}
                
                if (nrow(gaps)==0){
                  clst_found <- rbind(clst_found,res)
                } else {
                  gaps$pass <- apply(gaps,1,function(x){
                    num_mm <- strsplit(x[3],split = ',') %>% unlist %>% length
                    mm_size <- x[4] %>% as.numeric()
                    max_allowed_gs <- mm_size + (num_mm+1)*btwn_mt_max_gap
                    gs <- x[5] %>% as.numeric
                    pass_or_not <- (mm_size < gs) & (gs < max_allowed_gs)
                    return(pass_or_not)
                  })
                  # if (name %in% seq_uniq_in_meme) {print(gaps)}
                  if (all(gaps$pass)){
                    clst_found <- rbind(clst_found,res)
                  }
                }
                
                #  clst_found[[paste0(k,'_',l)]] <- list(res = res, gaps = gaps)
                #  l = l + 1
              }
            }
          }
          
          if (nrow(clst_found)>0){break}
        }
      }
    }
    return(clst_found)
  }, cl = ncores)
  return(clst_found)
}

In [3]:
wd <- "/Users/ninhle/Desktop/Research/scPASU_pipeline_runs/Ureter10_scPASU_run/outputs/differentiation_stage_cellranger_peakcount/"
setwd(wd)

In [4]:
# Variables
fprefix <- 'all_TU_disjoint_utr3'
# fprefix <- 'all_TU_disjoint_utr3_5kbflank'

seq_bed <- read.delim(paste0(fprefix,'.bed'),
                      header = F)
colnames(seq_bed) <- c('chr','start','end','seq_name','score','strand')
head(seq_bed)

,chr,start,end,seq_name,score,strand
,<chr>,<int>,<int>,<chr>,<int>,<chr>
1,chr1,70006,71586,TU4_UTR1,1,+
2,chr1,944151,944576,TU17_UTR2,1,+
3,chr1,965189,965720,TU18_UTR3,1,+
4,chr1,974573,975866,TU19_UTR4,1,+
5,chr1,1014476,1014541,TU20_UTR5,1,+
6,chr1,1054979,1056119,TU21_UTR6,1,+


##### Lengthening motif cluster detecting

In [10]:
#### SEA
filedir=paste0('sea_out_lgthn_',fprefix)
message(paste0('Running sea cluster detection for ',filedir,'...'))
  
sea_res <- read.delim(paste0(filedir,'/sites.tsv'))
sea_res <- sea_res[!sea_res$motif_ID=='#',]
sea_res$strand <- str_extract_all(sea_res$seq_ID, "\\(([^)]+)\\)") %>% unlist
sea_res$strand <- gsub("\\(|\\)","",sea_res$strand)
sea_res$cis <- sea_res$site_Strand==sea_res$strand
sea_res$seq_ID <- gsub("\\(([^)]+)\\)","",sea_res$seq_ID)
sea_res$chr <- seq_bed$chr[match(sea_res$seq_ID,seq_bed$seq_name)]
sea_res$range_start <- seq_bed$start[match(sea_res$seq_ID,seq_bed$seq_name)]
sea_res$range_end <- seq_bed$end[match(sea_res$seq_ID,seq_bed$seq_name)]
sea_res$site_width <- nchar(sea_res$site_Sequence)
sea_res$start <- ifelse(sea_res$strand=='+',sea_res$range_start+sea_res$site_Start-1,
                                                sea_res$range_end-sea_res$site_Start-sea_res$site_width+1)
sea_res$end <- sea_res$start+sea_res$site_width
colnames(sea_res)[c(2,3,4,5,6,8)] <- c('meme_name','seq_name','site_start','site_end','site_strand','site')
sea_res <- sea_res %>% select(seq_name,site_strand,site_start,site,meme_name,strand,cis,chr,
                                                      range_start,range_end,site_width,start,end)
sea_res <- sea_res %>% mutate(pval = NA,flank1 = NA,flank2 = NA) %>%
    relocate(pval, .after = site_start) %>%
    relocate(flank1, .after = pval) %>%
    relocate(flank2, .after = site)
top_nmotif <- sea_res %>% group_by(seq_name) %>% summarize(n=n()) %>% arrange(desc(n))
print(top_nmotif[1:10,])
write.table(sea_res,paste0(filedir,'/sea_res_processed_',fprefix,'.tsv'),
            sep = '\t',quote = F,row.names = F)                       
sea_clst_found <- find_motif_cluster_in_sea(sea_res,mc = c('MEME-3'='+','MEME-1'='+','MEME-2'='+','MEME-5'='-','MEME-4'='+'),
                                              min_nmotif=3,btwn_mt_max_gap=40,ncores = 1)
saveRDS(sea_clst_found,paste0(filedir,'/sea_clst_found_',fprefix,'.rds'))

Running sea cluster detection for sea_out_lgthn_all_TU_disjoint_utr3...



# A tibble: 10 × 2
   seq_name             n
   <chr>            <int>
 1 TU22647_UTR12934    88
 2 TU4659_UTR23842     79
 3 TU26609_UTR15674    75
 4 TU32685_UTR20212    69
 5 TU26145_UTR15390    63
 6 TU36069_UTR25042    63
 7 TU23086_UTR13233    59
 8 TU3786_UTR23194     59
 9 TU13708_UTR5877     56
10 TU418_UTR348        52
  |++++++++++++++++++++++++++++++++++++++++++++++++++| 100%


In [24]:
filedir=paste0('sea_out_lgthn_',fprefix)
sea_clst_found <- readRDS(file = paste0(filedir,'/sea_clst_found_',fprefix,'.rds'))
sea_clst_found_df <- do.call('rbind',sea_clst_found)
cat("Number of UTRs with motif cluster found:", length(unique(sea_clst_found_df$seq_name)), "\n")
head(sea_clst_found_df[order(sea_clst_found_df$min_consec_gap),])

Number of UTRs with motif cluster found: 4016 


,combination,seq_name,chr,strand,meme_start_site_start,meme_end_site_start,meme_seq,ref_motif,min_consec_gap
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>
TU1525_UTR1293,Orig_data,TU1525_UTR1293,chr1,+,2973,3125,"MEME-1,MEME-2,MEME-5,MEME-4","MEME-3,MEME-1,MEME-2,MEME-5,MEME-4",0
TU22463_UTR12756.1,Combination_MEME-2_16931,TU22463_UTR12756,chr12,+,1011,1270,"MEME-3,MEME-5,MEME-4","MEME-3,MEME-1,MEME-2,MEME-5,MEME-4",0
TU33615_UTR21115.1,Combination_MEME-2_16248,TU33615_UTR21115,chr19,-,8931,9084,"MEME-1,MEME-2,MEME-5,MEME-4","MEME-3,MEME-1,MEME-2,MEME-5,MEME-4",0
TU37452_UTR26142.3,Combination_MEME-2_12293_MEME-2_16043_MEME-2_18298,TU37452_UTR26142,chrX,+,861,1027,"MEME-4,MEME-5,MEME-2,MEME-1","MEME-4,MEME-5,MEME-2,MEME-1,MEME-3",0
TU37452_UTR26142.5,Combination_MEME-2_12293_MEME-3_20713_MEME-2_18298,TU37452_UTR26142,chrX,+,861,1113,"MEME-4,MEME-5,MEME-2,MEME-1,MEME-3","MEME-4,MEME-5,MEME-2,MEME-1,MEME-3",0
TU37452_UTR26142.7,Combination_MEME-3_21327_MEME-3_20713_MEME-2_18298,TU37452_UTR26142,chrX,+,861,1113,"MEME-4,MEME-5,MEME-2,MEME-1,MEME-3","MEME-4,MEME-5,MEME-2,MEME-1,MEME-3",0


In [12]:
sea_clst_found_genes <- unique(gsub('_UTR[0-9]+','',sea_clst_found_df$seq_name))

In [6]:
turef_lgthn_mc_genes <- read.delim('turef_lgthn_mc_genes.txt')
cat('Number of TUs with full or partial motif cluster from discover set: ',length(unique(turef_lgthn_mc_genes$tu)))

Number of TUs with full or partial motif cluster from discover set:  30

In [14]:
setdiff(unique(turef_lgthn_mc_genes$tu), sea_clst_found_genes)

[1] "TU12003" "TU14133" "TU17295" "TU17713" "TU18776" "TU19555" "TU25707"
 [8] "TU30807" "TU36726" "TU36855" "TU8157"

Full or partial motif cluster false negatives Lengthening: 'TU12003', 'TU14133', 'TU17295', 'TU17713', 'TU18776', 'TU19555', 'TU25707', 'TU30807', 'TU36726', 'TU36855', 'TU8157' 

Full or partial motif cluster fall negatives because cluster first detected in flanks

##### Lengthening motif cluster cleaning up

In [15]:
calculate_overlap <- function(start1, end1, start2, end2) {
  overlap_start <- max(start1, start2)
  overlap_end <- min(end1, end2)
  overlap_length <- max(0, overlap_end - overlap_start + 1)
  
  range1_length <- end1 - start1 + 1
  range2_length <- end2 - start2 + 1
  
  # Calculate overlap as percentage of the smaller range
  min_length <- min(range1_length, range2_length)
  overlap_pct <- overlap_length / min_length
  
  return(overlap_pct)
}

In [16]:
genes <- readRDS("/Users/ninhle/Desktop/Research/mcast_analyses/genes.rds")
ref <- genes$utr3 %>% as.data.frame()
head(ref)

Loading required namespace: GenomicRanges



,seqnames,start,end,width,strand,tu,gene.id,name,utr
,<fct>,<int>,<int>,<int>,<fct>,<chr>,<chr>,<chr>,<chr>
1,chr1,70006,71585,1580,+,TU4,ENSG00000186092,OR4F5,UTR1
2,chr1,944151,944575,425,+,TU17,ENSG00000187634,SAMD11,UTR2
3,chr1,965189,965719,531,+,TU18,ENSG00000187961,KLHL17,UTR3
4,chr1,974573,975865,1293,+,TU19,ENSG00000187583,PLEKHN1,UTR4
5,chr1,1014476,1014540,65,+,TU20,ENSG00000187608,ISG15,UTR5
6,chr1,1054979,1056118,1140,+,TU21,ENSG00000188157,AGRN,UTR6


In [17]:
## Group overlapping ranges within each seq_name
filedir=paste0('sea_out_lgthn_',fprefix)
sea_res <- read.delim(paste0(filedir,'/sea_res_processed_',fprefix,'.tsv'))
head(sea_res)
sea_clst_found <- readRDS(file = paste0(filedir,'/sea_clst_found_',fprefix,'.rds'))
sea_clst_found_df <- do.call('rbind',sea_clst_found)
sea_clst_grouped <- sea_clst_found_df %>%
  group_by(seq_name) %>%
  arrange(meme_start_site_start) %>%
  mutate(
    range_length = meme_end_site_start - meme_start_site_start + 1,
    overlap_group = {
      n_rows <- n()
      if (n_rows == 1) {
        1
      } else {
        groups <- rep(NA, n_rows)
        groups[1] <- 1
        current_group <- 1
        
        for (i in 2:n_rows) {
          # Check overlap with previous rows in current group
          overlaps_with_group <- FALSE
          for (j in 1:(i-1)) {
            if (groups[j] == current_group) {
              overlap_pct <- calculate_overlap(
                meme_start_site_start[i], meme_end_site_start[i],
                meme_start_site_start[j], meme_end_site_start[j]
              )
              if (overlap_pct >= 0.9) {
                overlaps_with_group <- TRUE
                break
              }
            }
          }
          
          if (overlaps_with_group) {
            groups[i] <- current_group
          } else {
            current_group <- current_group + 1
            groups[i] <- current_group
          }
        }
        groups
      }
    }
  ) %>%
  group_by(seq_name, overlap_group) %>%
  summarize(
    cluster_count = n(),
    all_combinations = paste(unique(combination), collapse = ";"),
    all_meme_seqs = paste(unique(meme_seq), collapse = ";"),
    all_starts = {
    unique_seqs <- unique(meme_seq)
    start_values <- sapply(unique_seqs, function(seq) meme_start_site_start[meme_seq == seq][1])
    list(start_values)},
    all_ends = {
    unique_seqs <- unique(meme_seq)
    end_values <- sapply(unique_seqs, function(seq) meme_end_site_start[meme_seq == seq][1])
    list(end_values)},
    .groups = "drop"
  ) %>%
  rowwise() %>%
  mutate(
    # Count motifs in each meme_seq
    meme_seqs_list = list(strsplit(all_meme_seqs, ";")[[1]]),
    motif_counts = list(sapply(meme_seqs_list, function(x) length(strsplit(x, ",")[[1]]))),
    max_motif_count = max(unlist(motif_counts)),
    
    # Find meme_seqs with maximum motif count
    max_meme_seqs = list(meme_seqs_list[motif_counts == max_motif_count]),
  
    # Extract final values directly
    meme_seqs = if(length(max_meme_seqs) == 1) {
    max_meme_seqs
    } else {
      paste(max_meme_seqs, collapse = ";")
    },
    full_motif_cluster = max_motif_count == 5,

    min_start = if(length(max_meme_seqs) == 1) {
      max_seq = max_meme_seqs
      seq_idx = which(meme_seqs_list == max_seq)
      all_starts[seq_idx]
    } else {
      max_indices = which(motif_counts == max_motif_count)
      min(all_starts[max_indices])},
    
    max_end = if(length(max_meme_seqs) == 1) {
      max_seq = max_meme_seqs
      seq_idx = which(meme_seqs_list == max_seq)
      all_ends[seq_idx]
    } else {
      max_indices = which(motif_counts == max_motif_count)
      max(all_ends[max_indices])
    },
  
  combinations = all_combinations
) %>%
  ungroup() %>%
  select(seq_name, overlap_group, cluster_count, full_motif_cluster, min_start, max_end, combinations, meme_seqs) %>%
  arrange(seq_name, min_start)
sea_clst_grouped$tu <- gsub('_UTR[0-9]+','',sea_clst_grouped$seq_name)
sea_clst_grouped$gene <- ref$name[match(sea_clst_grouped$tu,ref$tu)]

,seq_name,site_strand,site_start,pval,flank1,site,flank2,meme_name,strand,cis,chr,range_start,range_end,site_width,start,end
,<chr>,<chr>,<int>,<lgl>,<lgl>,<chr>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<int>,<int>,<int>,<int>,<int>
1,TU13400_UTR5644,+,653,NA,NA,TGCAGTGGCGCGATCTCGGCTCACTGCAACCT,NA,MEME-5,-,FALSE,chr6,116274858,116278520,32,116277836,116277868
2,TU17531_UTR8696,+,1803,NA,NA,TGCAGTGGCGCGATCTCGGCTCACTGCAACCT,NA,MEME-5,+,TRUE,chr9,124810410,124814886,32,124812212,124812244
3,TU15342_UTR7163,+,303,NA,NA,TGCAGTGGCGCGATCTCGGCTCACTGCAACCT,NA,MEME-5,-,FALSE,chr7,139043515,139047597,32,139047263,139047295
4,TU24917_UTR14518,-,640,NA,NA,TGCAGTGGCGCGATCTCGGCTCACTGCAACCT,NA,MEME-5,-,TRUE,chr13,113324845,113325550,32,113324879,113324911
5,TU23065_UTR13222,+,1965,NA,NA,TGCAGTGGCGCGATCTCGGCTCACTGCAACCT,NA,MEME-5,+,TRUE,chr12,131943584,131945897,32,131945548,131945580
6,TU8804_UTR28907,-,3429,NA,NA,TGCAGTGGCGCGATCTCGGCTCACTGCAACCT,NA,MEME-5,+,FALSE,chr4,127832872,127840734,32,127836300,127836332


In [18]:
sea_clst_grouped$strand <- seq_bed$strand[match(sea_clst_grouped$seq_name,seq_bed$seq_name)]
sea_clst_grouped$chr <- seq_bed$chr[match(sea_clst_grouped$seq_name,seq_bed$seq_name)]
sea_clst_grouped$range_start <- seq_bed$start[match(sea_clst_grouped$seq_name,seq_bed$seq_name)]
sea_clst_grouped$range_end <- seq_bed$end[match(sea_clst_grouped$seq_name,seq_bed$seq_name)]


sea_clst_grouped$most_upstream_motif_start <- apply(sea_clst_grouped,1,function(x){
  strand <- x['strand']
  seq_name <- as.character(x['seq_name'])
  min_start <- as.numeric(x['min_start'])
  chr <- x['chr']
  start <- sea_res[which(sea_res$seq_name==seq_name & sea_res$chr==chr & sea_res$site_start==min_start),'start'] 
  return(start)
})

sea_clst_grouped$most_upstream_motif <- apply(sea_clst_grouped,1,function(x){
  strand <- x['strand']
  seq_name <- as.character(x['seq_name'])
  min_start <- as.numeric(x['min_start'])
  chr <- x['chr']
  motif <- sea_res[which(sea_res$seq_name==seq_name & sea_res$chr==chr & sea_res$site_start==min_start),'meme_name']
  return(motif)
})

sea_clst_grouped$most_downstream_motif_start <- apply(sea_clst_grouped,1,function(x){
  seq_name <- as.character(x['seq_name'])
  max_end <- as.numeric(x['max_end'])
  chr <- x['chr']
  start <- sea_res[which(sea_res$seq_name==seq_name & sea_res$chr==chr & sea_res$site_start==max_end),'start']
  return(start)
})

sea_clst_grouped$most_downstream_motif <- apply(sea_clst_grouped,1,function(x){
  seq_name <- as.character(x['seq_name'])
  max_end <- as.numeric(x['max_end'])
  chr <- x['chr']
  motif <- sea_res[which(sea_res$seq_name==seq_name & sea_res$chr==chr & sea_res$site_start==max_end),'meme_name']
  return(motif)
})


sea_clst_grouped <- sea_clst_grouped %>%
  relocate(chr, .after = seq_name) %>%
  relocate(strand, .after = chr) %>%
  relocate(range_start, .after = strand) %>%
  relocate(range_end, .after = range_start) %>%
  relocate(meme_seqs, .after = range_end) %>%
  relocate(full_motif_cluster, .after = meme_seqs) %>%
  relocate(most_upstream_motif, .after = full_motif_cluster) %>%
  relocate(most_upstream_motif_start, .after = most_upstream_motif) %>%
  relocate(most_downstream_motif, .after = most_upstream_motif_start) %>%
  relocate(most_downstream_motif_start, .after = most_downstream_motif) %>%
  relocate(cluster_count, .after = most_downstream_motif_start) %>%
  relocate(tu, .before = seq_name) %>% 
  relocate(gene, .before = tu) %>% select(-c(combinations,cluster_count,overlap_group,min_start,max_end))

In [19]:
sea_clst_grouped <- sea_clst_grouped %>%
  # Extract TU and current UTR index for sorting
  mutate(
    tu_part = gsub('_UTR[0-9]+', '', seq_name),
    current_utr = as.numeric(gsub('.*_UTR([0-9]+)', '\\1', seq_name))
  ) %>%
  group_by(tu_part) %>%
  arrange(
    tu_part, 
    case_when(
      strand == '+' ~ range_start,
      strand == '-' ~ -range_end
    )
  ) %>%
  mutate(
    # Create unique UTR identifier based on coordinates
    # utr_coords = paste(range_start, range_end, sep = "_"),
    # Get unique coordinates in order
    unique_seqs = list(unique(seq_name)),
    # Assign index based on position of coordinates in unique list
    new_utr_index = case_when(
      length(seq_name) == 1 ~ 0,
      TRUE ~ match(seq_name, unique_seqs[[1]]) 
    ),
    # Create new seq_name with reindexed UTR
    seq_name = paste0(tu_part, '_UTR', new_utr_index)
  ) %>%
  ungroup() %>%
  select(-tu_part, -current_utr, -new_utr_index, -unique_seqs) %>%
  arrange(gene, tu, seq_name)

head(sea_clst_grouped)
write.table(sea_clst_grouped,
            file = paste0(filedir,'/','motif_cluster_detection_res_',fprefix,'.tsv'),
            sep = '\t',quote = F,row.names = F)

gene,tu,seq_name,chr,strand,range_start,range_end,meme_seqs,full_motif_cluster,most_upstream_motif,most_upstream_motif_start,most_downstream_motif,most_downstream_motif_start
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<lgl>,<chr>,<int>,<chr>,<int>
A2ML1,TU22196,TU22196_UTR1,chr12,+,8875461,8875815,"MEME-5,MEME-1,MEME-3",FALSE,MEME-5,8875487,MEME-3,8875701
A2ML1,TU22196,TU22196_UTR2,chr12,+,8876058,8876788,"MEME-3,MEME-1,MEME-5",FALSE,MEME-3,8876388,MEME-5,8876606
AAK1,TU5339,TU5339_UTR1,chr2,-,69481056,69482589,"MEME-5,MEME-2,MEME-1,MEME-3",FALSE,MEME-5,69482286,MEME-3,69482067
AAK1,TU5339,TU5339_UTR2,chr2,-,69457997,69475872,"MEME-3,MEME-1,MEME-2,MEME-5,MEME-4",TRUE,MEME-3,69459527,MEME-4,69459270
AAMDC,TU20683,TU20683_UTR0,chr11,+,77918097,77918433,"MEME-3,MEME-2,MEME-5,MEME-4",FALSE,MEME-3,77918146,MEME-4,77918387
AARD,TU16086,TU16086_UTR0,chr8,+,116942699,116944488,"MEME-3,MEME-1,MEME-2,MEME-5,MEME-4",TRUE,MEME-3,116942715,MEME-4,116942970


In [20]:
sea_res[grepl('^TU17139',sea_res$seq_name),]

,seq_name,site_strand,site_start,pval,flank1,site,flank2,meme_name,strand,cis,chr,range_start,range_end,site_width,start,end
,<chr>,<chr>,<int>,<lgl>,<lgl>,<chr>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<int>,<int>,<int>,<int>,<int>
1359,TU17139_UTR8420,-,1889,NA,NA,TGCAATGGCACGATCTCGGCTCACTGCAGCCT,NA,MEME-5,+,FALSE,chr9,32450109,32454770,32,32451997,32452029
24591,TU17139_UTR8420,+,1669,NA,NA,AACAGCCAGGCGCAGTGGCTCATGCCTATAATTCCATCAC,NA,MEME-3,+,TRUE,chr9,32450109,32454770,40,32451777,32451817
28865,TU17139_UTR8420,+,1748,NA,NA,GTTCTAGACCAGCCTGGCCAACATGGTGAAACCCCGTCTCTACTAAA,NA,MEME-1,+,TRUE,chr9,32450109,32454770,47,32451856,32451903
35297,TU17139_UTR8420,+,2816,NA,NA,GCCTGGGCAATAGAGTGAGACCCTATCAATCAATCACCACC,NA,MEME-4,+,TRUE,chr9,32450109,32454770,41,32452924,32452965


In [21]:
# sea_clst_grouped[sea_clst_grouped$strand=='-',]%>% group_by(tu) %>% summarize(n = n_distinct(seq_name)) %>% arrange(desc(n)) 
sea_clst_grouped[sea_clst_grouped$tu == 'TU17139',]

gene,tu,seq_name,chr,strand,range_start,range_end,meme_seqs,full_motif_cluster,most_upstream_motif,most_upstream_motif_start,most_downstream_motif,most_downstream_motif_start
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<lgl>,<chr>,<int>,<chr>,<int>
ACO1,TU17139,TU17139_UTR0,chr9,+,32450109,32454770,"MEME-3,MEME-1,MEME-5",FALSE,MEME-3,32451777,MEME-5,32451997


In [22]:
# Number of TUs with lengthening motif clusters
length(unique(sea_clst_grouped$tu))

[1] 3791

In [23]:
nrow(sea_clst_grouped)

[1] 4822

In [8]:
filedir=paste0('sea_out_lgthn_',fprefix)
sea_clst_grouped <- read.delim(paste0(filedir,'/','motif_cluster_detection_res_', fprefix, '.tsv'))
sea_clst_grouped_outside_discovery_set <- sea_clst_grouped[!(sea_clst_grouped$tu %in% turef_lgthn_mc_genes$tu),]
write.table(sea_clst_grouped_outside_discovery_set,
            file = paste0(filedir,'/','motif_cluster_detection_res_outside_discovery_set_',fprefix,'.tsv'),
            sep = '\t',quote = F,row.names = F)
head(sea_clst_grouped_outside_discovery_set)
cat('Number of full or partial lengthening motif cluster:', nrow(sea_clst_grouped), '\n')
cat('Number of full or partial lengthening motif cluster outside discovery set:', nrow(sea_clst_grouped_outside_discovery_set), '\n')
cat('Number of TUs with full or partial lengthening motif cluster:', length(unique(sea_clst_grouped$tu)), '\n')
cat('Number of TUs with full or partial lengthening motif cluster outside discovery set:', length(unique(sea_clst_grouped_outside_discovery_set$tu)), '\n')

,gene,tu,seq_name,chr,strand,range_start,range_end,meme_seqs,full_motif_cluster,most_upstream_motif,most_upstream_motif_start,most_downstream_motif,most_downstream_motif_start
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<lgl>,<chr>,<int>,<chr>,<int>
1,A2ML1,TU22196,TU22196_UTR1,chr12,+,8875461,8875815,"MEME-5,MEME-1,MEME-3",FALSE,MEME-5,8875487,MEME-3,8875701
2,A2ML1,TU22196,TU22196_UTR2,chr12,+,8876058,8876788,"MEME-3,MEME-1,MEME-5",FALSE,MEME-3,8876388,MEME-5,8876606
3,AAK1,TU5339,TU5339_UTR1,chr2,-,69481056,69482589,"MEME-5,MEME-2,MEME-1,MEME-3",FALSE,MEME-5,69482286,MEME-3,69482067
4,AAK1,TU5339,TU5339_UTR2,chr2,-,69457997,69475872,"MEME-3,MEME-1,MEME-2,MEME-5,MEME-4",TRUE,MEME-3,69459527,MEME-4,69459270
5,AAMDC,TU20683,TU20683_UTR0,chr11,+,77918097,77918433,"MEME-3,MEME-2,MEME-5,MEME-4",FALSE,MEME-3,77918146,MEME-4,77918387
6,AARD,TU16086,TU16086_UTR0,chr8,+,116942699,116944488,"MEME-3,MEME-1,MEME-2,MEME-5,MEME-4",TRUE,MEME-3,116942715,MEME-4,116942970


Number of full or partial lengthening motif cluster: 4822 
Number of full or partial lengthening motif cluster outside discovery set: 4797 
Number of TUs with full or partial lengthening motif cluster: 3791 
Number of TUs with full or partial lengthening motif cluster outside discovery set: 3772 


##### Shortening motif cluster detecting

In [25]:
#### SEA
filedir=paste0('sea_out_shrtn_',fprefix)
message(paste0('Running sea cluster detection for ',filedir,'...'))
  
sea_res <- read.delim(paste0(filedir,'/sites.tsv'))
sea_res <- sea_res[!sea_res$motif_ID=='#',]
sea_res$strand <- str_extract_all(sea_res$seq_ID, "\\(([^)]+)\\)") %>% unlist
sea_res$strand <- gsub("\\(|\\)","",sea_res$strand)
sea_res$cis <- sea_res$site_Strand==sea_res$strand
sea_res$seq_ID <- gsub("\\(([^)]+)\\)","",sea_res$seq_ID)
sea_res$chr <- seq_bed$chr[match(sea_res$seq_ID,seq_bed$seq_name)]
sea_res$range_start <- seq_bed$start[match(sea_res$seq_ID,seq_bed$seq_name)]
sea_res$range_end <- seq_bed$end[match(sea_res$seq_ID,seq_bed$seq_name)]
sea_res$site_width <- nchar(sea_res$site_Sequence)
sea_res$start <- ifelse(sea_res$strand=='+',sea_res$range_start+sea_res$site_Start-1,
                                                sea_res$range_end-sea_res$site_Start-sea_res$site_width+1)
sea_res$end <- sea_res$start+sea_res$site_width
colnames(sea_res)[c(2,3,4,5,6,8)] <- c('meme_name','seq_name','site_start','site_end','site_strand','site')
sea_res <- sea_res %>% select(seq_name,site_strand,site_start,site,meme_name,strand,cis,chr,
                                                      range_start,range_end,site_width,start,end)
sea_res <- sea_res %>% mutate(pval = NA,flank1 = NA,flank2 = NA) %>%
    relocate(pval, .after = site_start) %>%
    relocate(flank1, .after = pval) %>%
    relocate(flank2, .after = site)
top_nmotif <- sea_res %>% group_by(seq_name) %>% summarize(n=n()) %>% arrange(desc(n))
print(top_nmotif[1:20,])
write.table(sea_res,paste0(filedir,'/sea_res_processed_',fprefix,'.tsv'),
            sep = '\t',quote = F,row.names = F)   

# remove problematic seq
seq_names <- unique(sea_res$seq_name)
problematic_seq_idx <- 1518
# problematic_seq_idx <- which(seq_names %in% top_nmotif$seq_name[1:20])
cat('Removing problematic seq', seq_names[problematic_seq_idx], '\n')
subset_seq_names <- seq_names[-(problematic_seq_idx)]
sea_res <- sea_res[sea_res$seq_name %in% subset_seq_names,]

sea_clst_found <- find_motif_cluster_in_sea(sea_res,mc = c('MEME-1'='-','MEME-2'='-','MEME-4'='-','MEME-3'='+','MEME-5'='-'),
                                              min_nmotif=3,btwn_mt_max_gap=40,ncores = 1)
saveRDS(sea_clst_found,paste0(filedir,'/sea_clst_found_',fprefix,'.rds'))

Running sea cluster detection for sea_out_shrtn_all_TU_disjoint_utr3...



# A tibble: 20 × 2
   seq_name             n
   <chr>            <int>
 1 TU22647_UTR12934    97
 2 TU4659_UTR23842     76
 3 TU26609_UTR15674    73
 4 TU26145_UTR15390    68
 5 TU32685_UTR20212    62
 6 TU36069_UTR25042    59
 7 TU13708_UTR5877     56
 8 TU14230_UTR6295     51
 9 TU27563_UTR16332    51
10 TU3786_UTR23194     51
11 TU12396_UTR4853     50
12 TU13562_UTR5805     50
13 TU23086_UTR13233    50
14 TU418_UTR348        50
15 TU9714_UTR29577     49
16 TU22761_UTR12995    47
17 TU37991_UTR26666    47
18 TU1054_UTR867       46
19 TU33335_UTR20846    46
20 TU28128_UTR16802    45
Removing problematic seq TU22647_UTR12934 
  |++++++++++++++++++++++++++++++++++++++++++++++++++| 100%


In [45]:
filedir=paste0('sea_out_shrtn_',fprefix)
sea_res_processed <- read.delim(paste0(filedir,'/sea_res_processed_',fprefix,'.tsv'))
sea_res_problematic_seq <- sea_res_processed[sea_res_processed$seq_name %in% seq_names[problematic_seq_idx],]
print(sea_res_problematic_seq[order(sea_res_problematic_seq$site_start),-c(4:7)])

              seq_name site_strand site_start meme_name strand   cis   chr
8364  TU22647_UTR12934           -        108    MEME-1      + FALSE chr12
16113 TU22647_UTR12934           -        155    MEME-2      + FALSE chr12
34141 TU22647_UTR12934           -        224    MEME-4      + FALSE chr12
5200  TU22647_UTR12934           -        242    MEME-1      + FALSE chr12
22095 TU22647_UTR12934           +        275    MEME-3      +  TRUE chr12
31960 TU22647_UTR12934           -        341    MEME-5      + FALSE chr12
12335 TU22647_UTR12934           -        573    MEME-1      + FALSE chr12
25474 TU22647_UTR12934           +        606    MEME-3      +  TRUE chr12
6964  TU22647_UTR12934           -       1338    MEME-1      + FALSE chr12
33894 TU22647_UTR12934           -       1454    MEME-4      + FALSE chr12
6734  TU22647_UTR12934           -       1472    MEME-1      + FALSE chr12
22742 TU22647_UTR12934           +       1505    MEME-3      +  TRUE chr12
29782 TU22647_UTR12934   

In [26]:
filedir=paste0('sea_out_shrtn_',fprefix)
sea_clst_found <- readRDS(file = paste0(filedir,'/sea_clst_found_',fprefix,'.rds'))
sea_clst_found_df <- do.call('rbind',sea_clst_found)
cat("Number of UTRs with motif cluster found:", length(unique(sea_clst_found_df$seq_name)), "\n")
head(sea_clst_found_df[order(sea_clst_found_df$min_consec_gap),])

Number of UTRs with motif cluster found: 3624 


,combination,seq_name,chr,strand,meme_start_site_start,meme_end_site_start,meme_seq,ref_motif,min_consec_gap
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>
TU11334_UTR3934,Orig_data,TU11334_UTR3934,chr5,-,4405,4634,"MEME-1,MEME-2,MEME-4,MEME-3,MEME-5","MEME-1,MEME-2,MEME-4,MEME-3,MEME-5",0
TU12182_UTR4672,Combination_MEME-2_19553,TU12182_UTR4672,chr6,+,3161,3393,"MEME-5,MEME-3,MEME-4,MEME-2,MEME-1","MEME-5,MEME-3,MEME-4,MEME-2,MEME-1",0
TU15081_UTR6910,Orig_data,TU15081_UTR6910,chr7,-,827,1059,"MEME-5,MEME-3,MEME-4,MEME-2,MEME-1","MEME-5,MEME-3,MEME-4,MEME-2,MEME-1",0
TU16447_UTR7950.1,Combination_MEME-1_7238_MEME-1_12575,TU16447_UTR7950,chr8,-,238,468,"MEME-5,MEME-3,MEME-4,MEME-2,MEME-1","MEME-5,MEME-3,MEME-4,MEME-2,MEME-1",0
TU16447_UTR7950.2,Combination_MEME-3_25394_MEME-1_12575,TU16447_UTR7950,chr8,-,238,468,"MEME-5,MEME-3,MEME-4,MEME-2,MEME-1","MEME-5,MEME-3,MEME-4,MEME-2,MEME-1",0
TU16447_UTR7950.3,Combination_MEME-4_34833_MEME-1_12575,TU16447_UTR7950,chr8,-,238,468,"MEME-5,MEME-3,MEME-4,MEME-2,MEME-1","MEME-5,MEME-3,MEME-4,MEME-2,MEME-1",0


In [27]:
sea_clst_found_genes <- unique(gsub('_UTR[0-9]+','',sea_clst_found_df$seq_name))

In [9]:
turef_shtn_mc_genes <- read.delim('turef_shtn_mc_genes.txt')
cat('Number of TUs with full or partial motif cluster from discover set: ',length(unique(turef_shtn_mc_genes$tu)))

Number of TUs with full or partial motif cluster from discover set:  29

In [29]:
setdiff(unique(turef_shtn_mc_genes$tu), sea_clst_found_genes)

[1] "TU1169"  "TU26512" "TU3179"  "TU34842" "TU5795"


Full or partial motif cluster false negatives Shortening: 'TU1169', 'TU26512', 'TU3179', 'TU34842', 'TU5795' (none were picked up even when relax min_nmotif to 2)

Full or partial motif cluster fall negatives because cluster first detected in flanks

#### Shortening motif cluster cleaning up

In [30]:
calculate_overlap <- function(start1, end1, start2, end2) {
  overlap_start <- max(start1, start2)
  overlap_end <- min(end1, end2)
  overlap_length <- max(0, overlap_end - overlap_start + 1)
  
  range1_length <- end1 - start1 + 1
  range2_length <- end2 - start2 + 1
  
  # Calculate overlap as percentage of the smaller range
  min_length <- min(range1_length, range2_length)
  overlap_pct <- overlap_length / min_length
  
  return(overlap_pct)
}

In [31]:
genes <- readRDS("/Users/ninhle/Desktop/Research/mcast_analyses/genes.rds")
ref <- genes$utr3 %>% as.data.frame()
head(ref)

,seqnames,start,end,width,strand,tu,gene.id,name,utr
,<fct>,<int>,<int>,<int>,<fct>,<chr>,<chr>,<chr>,<chr>
1,chr1,70006,71585,1580,+,TU4,ENSG00000186092,OR4F5,UTR1
2,chr1,944151,944575,425,+,TU17,ENSG00000187634,SAMD11,UTR2
3,chr1,965189,965719,531,+,TU18,ENSG00000187961,KLHL17,UTR3
4,chr1,974573,975865,1293,+,TU19,ENSG00000187583,PLEKHN1,UTR4
5,chr1,1014476,1014540,65,+,TU20,ENSG00000187608,ISG15,UTR5
6,chr1,1054979,1056118,1140,+,TU21,ENSG00000188157,AGRN,UTR6


In [32]:
## Group overlapping ranges within each seq_name
filedir=paste0('sea_out_shrtn_',fprefix)
sea_res <- read.delim(paste0(filedir,'/sea_res_processed_',fprefix,'.tsv'))
head(sea_res)
sea_clst_found <- readRDS(file = paste0(filedir,'/sea_clst_found_',fprefix,'.rds'))
sea_clst_found_df <- do.call('rbind',sea_clst_found)
sea_clst_grouped <- sea_clst_found_df %>%
  group_by(seq_name) %>%
  arrange(meme_start_site_start) %>%
  mutate(
    range_length = meme_end_site_start - meme_start_site_start + 1,
    overlap_group = {
      n_rows <- n()
      if (n_rows == 1) {
        1
      } else {
        groups <- rep(NA, n_rows)
        groups[1] <- 1
        current_group <- 1
        
        for (i in 2:n_rows) {
          # Check overlap with previous rows in current group
          overlaps_with_group <- FALSE
          for (j in 1:(i-1)) {
            if (groups[j] == current_group) {
              overlap_pct <- calculate_overlap(
                meme_start_site_start[i], meme_end_site_start[i],
                meme_start_site_start[j], meme_end_site_start[j]
              )
              if (overlap_pct >= 0.9) {
                overlaps_with_group <- TRUE
                break
              }
            }
          }
          
          if (overlaps_with_group) {
            groups[i] <- current_group
          } else {
            current_group <- current_group + 1
            groups[i] <- current_group
          }
        }
        groups
      }
    }
  ) %>%
  group_by(seq_name, overlap_group) %>%
  summarize(
    cluster_count = n(),
    all_combinations = paste(unique(combination), collapse = ";"),
    all_meme_seqs = paste(unique(meme_seq), collapse = ";"),
    all_starts = {
    unique_seqs <- unique(meme_seq)
    start_values <- sapply(unique_seqs, function(seq) meme_start_site_start[meme_seq == seq][1])
    list(start_values)},
    all_ends = {
    unique_seqs <- unique(meme_seq)
    end_values <- sapply(unique_seqs, function(seq) meme_end_site_start[meme_seq == seq][1])
    list(end_values)},
    .groups = "drop"
  ) %>%
  rowwise() %>%
  mutate(
    # Count motifs in each meme_seq
    meme_seqs_list = list(strsplit(all_meme_seqs, ";")[[1]]),
    motif_counts = list(sapply(meme_seqs_list, function(x) length(strsplit(x, ",")[[1]]))),
    max_motif_count = max(unlist(motif_counts)),
    
    # Find meme_seqs with maximum motif count
    max_meme_seqs = list(meme_seqs_list[motif_counts == max_motif_count]),
  
    # Extract final values directly
    meme_seqs = if(length(max_meme_seqs) == 1) {
    max_meme_seqs
    } else {
      paste(max_meme_seqs, collapse = ";")
    },
    full_motif_cluster = max_motif_count == 5,

    min_start = if(length(max_meme_seqs) == 1) {
      max_seq = max_meme_seqs
      seq_idx = which(meme_seqs_list == max_seq)
      all_starts[seq_idx]
    } else {
      max_indices = which(motif_counts == max_motif_count)
      min(all_starts[max_indices])},
    
    max_end = if(length(max_meme_seqs) == 1) {
      max_seq = max_meme_seqs
      seq_idx = which(meme_seqs_list == max_seq)
      all_ends[seq_idx]
    } else {
      max_indices = which(motif_counts == max_motif_count)
      max(all_ends[max_indices])
    },
  
  combinations = all_combinations
) %>%
  ungroup() %>%
  select(seq_name, overlap_group, cluster_count, full_motif_cluster, min_start, max_end, combinations, meme_seqs) %>%
  arrange(seq_name, min_start)
sea_clst_grouped$tu <- gsub('_UTR[0-9]+','',sea_clst_grouped$seq_name)
sea_clst_grouped$gene <- ref$name[match(sea_clst_grouped$tu,ref$tu)]

,seq_name,site_strand,site_start,pval,flank1,site,flank2,meme_name,strand,cis,chr,range_start,range_end,site_width,start,end
,<chr>,<chr>,<int>,<lgl>,<lgl>,<chr>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<int>,<int>,<int>,<int>,<int>
1,TU3972_UTR23332,-,1579,NA,NA,GATCTGCCTGCCTCGGCCTCCCAAAGTGCTGGGCTTACAAG,NA,MEME-1,+,FALSE,chr2,69879002,69881385,41,69880580,69880621
2,TU4470_UTR23681,+,2122,NA,NA,GATCCGCCCGCCTCGGCCTCCCAAAGTGCTGGGATTACAAG,NA,MEME-1,+,TRUE,chr2,167870018,167874046,41,167872139,167872180
3,TU27298_UTR16133,-,890,NA,NA,GATCCGCCCGCCTCGGCCTCCCAAAGTGCTGGGATTACAAG,NA,MEME-1,-,TRUE,chr15,52115100,52122760,41,52121830,52121871
4,TU483_UTR400,+,759,NA,NA,GATCCGCCCGCCTCGGCCTCCCAAAGTGCTGGGATTACAAG,NA,MEME-1,+,TRUE,chr1,42798223,42800828,41,42798981,42799022
5,TU18362_UTR9436,-,4039,NA,NA,GATCCGCCCGCCTCGGCCTCCCAAAGTGCTGGGATTACAAG,NA,MEME-1,-,TRUE,chr9,134903232,134909801,41,134905722,134905763
6,TU19025_UTR9914,+,733,NA,NA,GATCCGCCCGCCTCGGCCTCCCAAAGTGCTGGGATTACAAG,NA,MEME-1,+,TRUE,chr10,102900698,102901900,41,102901430,102901471


In [33]:
sea_clst_grouped$strand <- seq_bed$strand[match(sea_clst_grouped$seq_name,seq_bed$seq_name)]
sea_clst_grouped$chr <- seq_bed$chr[match(sea_clst_grouped$seq_name,seq_bed$seq_name)]
sea_clst_grouped$range_start <- seq_bed$start[match(sea_clst_grouped$seq_name,seq_bed$seq_name)]
sea_clst_grouped$range_end <- seq_bed$end[match(sea_clst_grouped$seq_name,seq_bed$seq_name)]


sea_clst_grouped$most_upstream_motif_start <- apply(sea_clst_grouped,1,function(x){
  strand <- x['strand']
  seq_name <- as.character(x['seq_name'])
  min_start <- as.numeric(x['min_start'])
  chr <- x['chr']
  start <- sea_res[which(sea_res$seq_name==seq_name & sea_res$chr==chr & sea_res$site_start==min_start),'start'] 
  return(start)
})

sea_clst_grouped$most_upstream_motif <- apply(sea_clst_grouped,1,function(x){
  strand <- x['strand']
  seq_name <- as.character(x['seq_name'])
  min_start <- as.numeric(x['min_start'])
  chr <- x['chr']
  motif <- sea_res[which(sea_res$seq_name==seq_name & sea_res$chr==chr & sea_res$site_start==min_start),'meme_name']
  return(motif)
})

sea_clst_grouped$most_downstream_motif_start <- apply(sea_clst_grouped,1,function(x){
  seq_name <- as.character(x['seq_name'])
  max_end <- as.numeric(x['max_end'])
  chr <- x['chr']
  start <- sea_res[which(sea_res$seq_name==seq_name & sea_res$chr==chr & sea_res$site_start==max_end),'start']
  return(start)
})

sea_clst_grouped$most_downstream_motif <- apply(sea_clst_grouped,1,function(x){
  seq_name <- as.character(x['seq_name'])
  max_end <- as.numeric(x['max_end'])
  chr <- x['chr']
  motif <- sea_res[which(sea_res$seq_name==seq_name & sea_res$chr==chr & sea_res$site_start==max_end),'meme_name']
  return(motif)
})


sea_clst_grouped <- sea_clst_grouped %>%
  relocate(chr, .after = seq_name) %>%
  relocate(strand, .after = chr) %>%
  relocate(range_start, .after = strand) %>%
  relocate(range_end, .after = range_start) %>%
  relocate(meme_seqs, .after = range_end) %>%
  relocate(full_motif_cluster, .after = meme_seqs) %>%
  relocate(most_upstream_motif, .after = full_motif_cluster) %>%
  relocate(most_upstream_motif_start, .after = most_upstream_motif) %>%
  relocate(most_downstream_motif, .after = most_upstream_motif_start) %>%
  relocate(most_downstream_motif_start, .after = most_downstream_motif) %>%
  relocate(cluster_count, .after = most_downstream_motif_start) %>%
  relocate(tu, .before = seq_name) %>% 
  relocate(gene, .before = tu) %>% select(-c(combinations,cluster_count,overlap_group,min_start,max_end))

In [34]:
sea_clst_grouped <- sea_clst_grouped %>%
  # Extract TU and current UTR index for sorting
  mutate(
    tu_part = gsub('_UTR[0-9]+', '', seq_name),
    current_utr = as.numeric(gsub('.*_UTR([0-9]+)', '\\1', seq_name))
  ) %>%
  group_by(tu_part) %>%
  arrange(
    tu_part, 
    case_when(
      strand == '+' ~ range_start,
      strand == '-' ~ -range_end
    )
  ) %>%
  mutate(
    # Create unique UTR identifier
    unique_seqs = list(unique(seq_name)),
    # Assign index 
    new_utr_index = case_when(
      length(seq_name) == 1 ~ 0,
      TRUE ~ match(seq_name, unique_seqs[[1]]) 
    ),
    # Create new seq_name with reindexed UTR
    seq_name = paste0(tu_part, '_UTR', new_utr_index)
  ) %>%
  ungroup() %>%
  select(-tu_part, -current_utr, -new_utr_index, -unique_seqs) %>%
  arrange(gene, tu, seq_name)

head(sea_clst_grouped)
write.table(sea_clst_grouped,
            file = paste0(filedir,'/','motif_cluster_detection_res_',fprefix,'.tsv'),
            sep = '\t',quote = F,row.names = F)

gene,tu,seq_name,chr,strand,range_start,range_end,meme_seqs,full_motif_cluster,most_upstream_motif,most_upstream_motif_start,most_downstream_motif,most_downstream_motif_start
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<lgl>,<chr>,<int>,<chr>,<int>
A1BG,TU34454,TU34454_UTR0,chr19,-,58345183,58347025,"MEME-5,MEME-3,MEME-1",FALSE,MEME-5,58345641,MEME-1,58345403
A2ML1,TU22196,TU22196_UTR1,chr12,+,8875461,8875815,"MEME-3,MEME-2,MEME-1",FALSE,MEME-3,8875503,MEME-1,8875676
A2ML1,TU22196,TU22196_UTR2,chr12,+,8876058,8876788,"MEME-1,MEME-2,MEME-3",FALSE,MEME-1,8876412,MEME-3,8876579
AAK1,TU5339,TU5339_UTR1,chr2,-,69481056,69482589,"MEME-5,MEME-3,MEME-2,MEME-1;MEME-5,MEME-4,MEME-2,MEME-1",FALSE,MEME-5,69482325,MEME-1,69482091
AAK1,TU5339,TU5339_UTR2,chr2,-,69457997,69475872,"MEME-5,MEME-3,MEME-2,MEME-1",FALSE,MEME-5,69462648,MEME-1,69462420
AARD,TU16086,TU16086_UTR0,chr8,+,116942699,116944488,"MEME-1,MEME-2,MEME-3,MEME-5",FALSE,MEME-1,116942739,MEME-5,116942971


In [35]:
# Number of TUs with shortening motif clusters
length(unique(sea_clst_grouped$tu))

[1] 3443

In [10]:
filedir=paste0('sea_out_shrtn_',fprefix)
sea_clst_grouped <- read.delim(paste0(filedir,'/','motif_cluster_detection_res_', fprefix, '.tsv'))
sea_clst_grouped_outside_discovery_set <- sea_clst_grouped[!(sea_clst_grouped$tu %in% turef_shtn_mc_genes$tu),]
write.table(sea_clst_grouped_outside_discovery_set,
            file = paste0(filedir,'/','motif_cluster_detection_res_outside_discovery_set_',fprefix,'.tsv'),
            sep = '\t',quote = F,row.names = F)
head(sea_clst_grouped_outside_discovery_set)
cat('Number of full or partial shortening motif cluster:', nrow(sea_clst_grouped), '\n')
cat('Number of full or partial shortening motif cluster outside discovery set:', nrow(sea_clst_grouped_outside_discovery_set), '\n')
cat('Number of TUs with full or partial shortening motif cluster:', length(unique(sea_clst_grouped$tu)), '\n')
cat('Number of TUs with full or partial shortening motif cluster outside discovery set:', length(unique(sea_clst_grouped_outside_discovery_set$tu)), '\n')

,gene,tu,seq_name,chr,strand,range_start,range_end,meme_seqs,full_motif_cluster,most_upstream_motif,most_upstream_motif_start,most_downstream_motif,most_downstream_motif_start
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<lgl>,<chr>,<int>,<chr>,<int>
1,A1BG,TU34454,TU34454_UTR0,chr19,-,58345183,58347025,"MEME-5,MEME-3,MEME-1",FALSE,MEME-5,58345641,MEME-1,58345403
2,A2ML1,TU22196,TU22196_UTR1,chr12,+,8875461,8875815,"MEME-3,MEME-2,MEME-1",FALSE,MEME-3,8875503,MEME-1,8875676
3,A2ML1,TU22196,TU22196_UTR2,chr12,+,8876058,8876788,"MEME-1,MEME-2,MEME-3",FALSE,MEME-1,8876412,MEME-3,8876579
4,AAK1,TU5339,TU5339_UTR1,chr2,-,69481056,69482589,"MEME-5,MEME-3,MEME-2,MEME-1;MEME-5,MEME-4,MEME-2,MEME-1",FALSE,MEME-5,69482325,MEME-1,69482091
5,AAK1,TU5339,TU5339_UTR2,chr2,-,69457997,69475872,"MEME-5,MEME-3,MEME-2,MEME-1",FALSE,MEME-5,69462648,MEME-1,69462420
6,AARD,TU16086,TU16086_UTR0,chr8,+,116942699,116944488,"MEME-1,MEME-2,MEME-3,MEME-5",FALSE,MEME-1,116942739,MEME-5,116942971


Number of full or partial shortening motif cluster: 3722 
Number of full or partial shortening motif cluster outside discovery set: 3695 
Number of TUs with full or partial shortening motif cluster: 3443 
Number of TUs with full or partial shortening motif cluster outside discovery set: 3419 
